In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pyperclip

/Users/enriccogemha/Developer/courses-at-Insper/social_networks/aps03/web_scrapper/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# set up the PATHs
URL_WEBSITE = 'https://limitlesstcg.com/tournaments?time=all&type=all&format=all&region=all&show=100'
URL_WEBSITE_TEST = 'https://limitlesstcg.com/tournaments?time=1months&type=all&format=standard&region=all&show=100'

In [3]:
# I need to use selenium to scrape the data from the website below
# https://limitlesstcg.com/tournaments?time=12months&type=all&format=standard&region=all&show=100
# And extract the decklists from the top 8 players of each tournament and save them in a csv file
# The csv file should have the following columns:
# Tournament Name, Date, Player Name, Deck Name, Card1, Card2, ..., Card60
# The csv file should have a row for each decklist
# begin now


# Initialize the WebDriver (in this case, for Chrome)
driver = webdriver.Chrome()

# Go to the limitlesstcg website
driver.get(URL_WEBSITE)

# Get all the urls for the tournaments

# all elements have a tag <a> and the href="/tournament/xxxxx" where xxxxx is the tournament id
# I can use the xpath to find all the elements with the tag <a> and then extract the href attribute
# wait for the page to load

urls = []
for page_index in range(1, 5):
    time.sleep(0.01)
    elements = driver.find_elements(By.XPATH, '/html/body/main/div/table/tbody//td[3]/a')
    for element in elements:
        urls.append(element.get_attribute('href'))
    button = driver.find_element(By.XPATH, '/html/body/main/div/div[2]/ul/li[6]')
    button.click()

print(len(urls))
print(urls[0])

353
https://limitlesstcg.com/tournaments/385


In [4]:
deck_urls = []

for url in urls:
    time.sleep(0.01)
    driver.get(url)
    # get the link for the decks of the top 8 players
    elements = driver.find_elements(By.XPATH, '/html/body/main/div/table/tbody//td[4]/a')
    i = 0
    for element in elements:
        deck_url = element.get_attribute('href') + '/cards'
        if len(element.get_attribute('href').split('?')) == 2:
            deck_url = (element.get_attribute('href').split('?')[0] + '/cards?' + element.get_attribute('href').split('?')[1])
        deck_urls.append(deck_url)
        i += 1
        if i == 8:
            break

print(len(deck_urls))
print(deck_urls)

2815
['https://limitlesstcg.com/decks/264/cards', 'https://limitlesstcg.com/decks/240/cards?variant=1', 'https://limitlesstcg.com/decks/264/cards', 'https://limitlesstcg.com/decks/264/cards?variant=1', 'https://limitlesstcg.com/decks/212/cards?variant=38', 'https://limitlesstcg.com/decks/264/cards?variant=1', 'https://limitlesstcg.com/decks/255/cards', 'https://limitlesstcg.com/decks/264/cards', 'https://limitlesstcg.com/decks/224/cards?variant=2', 'https://limitlesstcg.com/decks/255/cards', 'https://limitlesstcg.com/decks/264/cards?variant=3', 'https://limitlesstcg.com/decks/247/cards', 'https://limitlesstcg.com/decks/255/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/257/cards?variant=2', 'https://limitlesstcg.com/decks/211/cards?variant=1', 'https://limitlesstcg.com/decks/247/cards', 'https://limitlesstcg.com/decks/2

In [5]:
def get_set_codes_and_numbers():
    # Getting the a full list of set codes and card amounts (numbers) ever printed
    all_set_codes = []
    all_set_numbers = []
    driver.get('https://limitlesstcg.com/cards')
    time.sleep(1)
    set_codes = driver.find_elements(By.XPATH, '/html/body/main/div/table/tbody//td[1]/a/span')
    set_numbers = driver.find_elements(By.XPATH, '/html/body/main/div/table/tbody//td[3]/a')

    if len(set_codes) != len(set_numbers):
        print('Error: the number of set codes and set numbers is different')
        return None, None

    # then get the content of the element
    for i in range(len(set_codes)):
        set_code, set_number = set_codes[i], set_numbers[i]
        all_set_codes.append(set_code.text)
        all_set_numbers.append(int(set_number.text.split('\n')[0]))

    return all_set_codes, all_set_numbers

In [6]:
def get_clean_decklist(data):
    try:
        # Split the decklist into Pokemon, Trainer and Energy. Then remove Energy.
        card_list = data.split('\n\n')[:2]
        # Remove the first element of each list and separates the cards.
        card_list = [x.split('\n')[1:] for x in card_list]
        # join the both lists (append one into other)
        card_list = card_list[0] + card_list[1]
        # split each element by the ' '
        card_list = [x.split(' ') for x in card_list]
        # get the last two elements of each list
        card_list = [x[-2:] for x in card_list]
        # if the second element has just 2 characters, put a 0 in front of it
        card_list = [[x[0], '0' + x[1]] if len(x[1]) == 2 else x for x in card_list]
        # join the elements of each list (append one into other)
        card_list = [x[0] + x[1] for x in card_list]
    except:
        return None
    return card_list

In [8]:
i = 0
with open(f'output/decks.csv', 'w') as f:
    # write the header
    f.write('deck_id,card_id\n')
error_counter = 0
for deck_url in deck_urls:
    driver.get(deck_url)
    time.sleep(1)
    try:
    # get the button by class='export'
        button = driver.find_element(By.CLASS_NAME, 'export')
        button.click()
    except:
        error_counter += 1
        continue
    copied_text = pyperclip.paste()
    # save the text in a file
    with open(f'output/decks.csv', 'a') as f:
        clean_decklist = get_clean_decklist(copied_text)
        if clean_decklist is None:
            error_counter += 1
            continue
        for card in clean_decklist:
            f.write(f'{i},{card}\n')
    i += 1
print(f'Error: {error_counter}')

Error: 188


In [ ]:
driver.quit()